# Basic Exploratory Analysis

## 1. Read data

Because the dataset is massive, we will only focus on Manhattan.

In [1]:
import pandas as pd
import numpy as np

In [2]:
crashes = pd.read_csv('../data/raw_data/Motor_Vehicle_Collisions_-_Crashes.csv')

C:\Users\Manlaigiin\AppData\Local\Temp\ipykernel_8396\3657282000.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crashes = pd.read_csv('../data/raw_data/Motor_Vehicle_Collisions_-_Crashes.csv')


In [3]:
collision_ids = crashes.loc[crashes['BOROUGH'] == 'MANHATTAN', 'COLLISION_ID']
crashes = crashes.query("COLLISION_ID in @collision_ids")

In [4]:
crashes.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')

In [5]:
vehicles = pd.read_csv('../data/raw_data/Motor_Vehicle_Collisions_-_Vehicles.csv')

C:\Users\Manlaigiin\AppData\Local\Temp\ipykernel_8396\355120935.py:1: DtypeWarning: Columns (0,8) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles = pd.read_csv('../data/raw_data/Motor_Vehicle_Collisions_-_Vehicles.csv')


In [6]:
vehicles = vehicles.query("COLLISION_ID in @collision_ids")

In [7]:
vehicles.columns

Index(['UNIQUE_ID', 'COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'VEHICLE_ID',
       'STATE_REGISTRATION', 'VEHICLE_TYPE', 'VEHICLE_MAKE', 'VEHICLE_MODEL',
       'VEHICLE_YEAR', 'TRAVEL_DIRECTION', 'VEHICLE_OCCUPANTS', 'DRIVER_SEX',
       'DRIVER_LICENSE_STATUS', 'DRIVER_LICENSE_JURISDICTION', 'PRE_CRASH',
       'POINT_OF_IMPACT', 'VEHICLE_DAMAGE', 'VEHICLE_DAMAGE_1',
       'VEHICLE_DAMAGE_2', 'VEHICLE_DAMAGE_3', 'PUBLIC_PROPERTY_DAMAGE',
       'PUBLIC_PROPERTY_DAMAGE_TYPE', 'CONTRIBUTING_FACTOR_1',
       'CONTRIBUTING_FACTOR_2'],
      dtype='object')

In [8]:
persons = pd.read_csv('../data/raw_data/Motor_Vehicle_Collisions_-_Person.csv')
persons = persons.query("COLLISION_ID in @collision_ids")

The column names of `crashes` has empty space as word separators while the other two has underscore as separators. Let's first normalize it to underscores.

In [9]:
crashes.columns = [c.replace(' ', '_') for c in crashes]

### Check for common columns

In [10]:
crashes.columns.intersection(vehicles.columns)

Index(['CRASH_DATE', 'CRASH_TIME', 'COLLISION_ID'], dtype='object')

## 2. Merge the datasets on matching keys (and common columns)

The following is the schema for the three datasets.

![image.png](../images/schema.png)

In [11]:
common_cols = ['CRASH_DATE', 'CRASH_TIME', 'COLLISION_ID']

df = (
    crashes
    .merge(vehicles, on=common_cols)
#    .merge(persons, left_on=[*common_cols, 'UNIQUE_ID'], right_on=[*common_cols, 'VEHICLE_ID'])
)

In [12]:
df.shape

(558724, 51)

In [13]:
df.columns

Index(['CRASH_DATE', 'CRASH_TIME', 'BOROUGH', 'ZIP_CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON_STREET_NAME', 'CROSS_STREET_NAME',
       'OFF_STREET_NAME', 'NUMBER_OF_PERSONS_INJURED',
       'NUMBER_OF_PERSONS_KILLED', 'NUMBER_OF_PEDESTRIANS_INJURED',
       'NUMBER_OF_PEDESTRIANS_KILLED', 'NUMBER_OF_CYCLIST_INJURED',
       'NUMBER_OF_CYCLIST_KILLED', 'NUMBER_OF_MOTORIST_INJURED',
       'NUMBER_OF_MOTORIST_KILLED', 'CONTRIBUTING_FACTOR_VEHICLE_1',
       'CONTRIBUTING_FACTOR_VEHICLE_2', 'CONTRIBUTING_FACTOR_VEHICLE_3',
       'CONTRIBUTING_FACTOR_VEHICLE_4', 'CONTRIBUTING_FACTOR_VEHICLE_5',
       'COLLISION_ID', 'VEHICLE_TYPE_CODE_1', 'VEHICLE_TYPE_CODE_2',
       'VEHICLE_TYPE_CODE_3', 'VEHICLE_TYPE_CODE_4', 'VEHICLE_TYPE_CODE_5',
       'UNIQUE_ID', 'VEHICLE_ID', 'STATE_REGISTRATION', 'VEHICLE_TYPE',
       'VEHICLE_MAKE', 'VEHICLE_MODEL', 'VEHICLE_YEAR', 'TRAVEL_DIRECTION',
       'VEHICLE_OCCUPANTS', 'DRIVER_SEX', 'DRIVER_LICENSE_STATUS',
       'DRIVER_LICENSE_JURIS

## 3. Handling duplicate columns

In [14]:
df.loc[:, df.columns.str.contains('_x$|_y$')].head(3)

""
0
1
2


First, drop ID columns. Since we have `Latitude` and `Longitude` data, `Location` is redundant.

In [15]:
to_drop = [
    'BOROUGH', 'LOCATION', 'ZIP_CODE', 
    'ON_STREET_NAME', 'CROSS_STREET_NAME', 'OFF_STREET_NAME',
    'VEHICLE_ID', 'UNIQUE_ID',
    'VEHICLE_MODEL'
]

df.drop(columns=to_drop, inplace=True)

In [16]:
df.shape

(558724, 42)

In [17]:
df.filter(like='CONTRIBUTING').head(3)

,CONTRIBUTING_FACTOR_VEHICLE_1,CONTRIBUTING_FACTOR_VEHICLE_2,CONTRIBUTING_FACTOR_VEHICLE_3,CONTRIBUTING_FACTOR_VEHICLE_4,CONTRIBUTING_FACTOR_VEHICLE_5,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
0,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,Passing or Lane Usage Improper,Driver Inattention/Distraction
1,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,Unspecified,Unspecified
2,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.filter(like='CONTRIBUTING').isna().mean()

CONTRIBUTING_FACTOR_VEHICLE_1    0.002769
CONTRIBUTING_FACTOR_VEHICLE_2    0.096030
CONTRIBUTING_FACTOR_VEHICLE_3    0.945376
CONTRIBUTING_FACTOR_VEHICLE_4    0.987142
CONTRIBUTING_FACTOR_VEHICLE_5    0.995704
CONTRIBUTING_FACTOR_1            0.025995
CONTRIBUTING_FACTOR_2            0.554768
dtype: float64

In [19]:
df.filter(regex='^VEHICLE_TYPE').head(3)

,VEHICLE_TYPE_CODE_1,VEHICLE_TYPE_CODE_2,VEHICLE_TYPE_CODE_3,VEHICLE_TYPE_CODE_4,VEHICLE_TYPE_CODE_5,VEHICLE_TYPE
0,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN
1,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,Station Wagon/Sport Utility Vehicle
2,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN


In [20]:
df.filter(regex='^VEHICLE_TYPE').isna().mean()

VEHICLE_TYPE_CODE_1    0.003538
VEHICLE_TYPE_CODE_2    0.098616
VEHICLE_TYPE_CODE_3    0.945925
VEHICLE_TYPE_CODE_4    0.987305
VEHICLE_TYPE_CODE_5    0.995755
VEHICLE_TYPE           0.028902
dtype: float64

In [21]:
df.filter(regex='^VEHICLE_DAMAGE').head(3)

,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3
0,No Damage,NaN,NaN,NaN
1,Other,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN


In [22]:
df.filter(regex='^VEHICLE_DAMAGE').isna().mean()

VEHICLE_DAMAGE      0.558723
VEHICLE_DAMAGE_1    0.732041
VEHICLE_DAMAGE_2    0.785264
VEHICLE_DAMAGE_3    0.811555
dtype: float64

All these columns basically record the same thing, so we'll only keep the one colmn and drop the rest.

In [23]:
df['VEHICLE_TYPE_CODE'] = df.filter(regex='^VEHICLE_TYPE').bfill(axis=1).iloc[:, 0]
df['CONTRIBUTING_FACTOR'] = df.filter(like='CONTRIBUTING').bfill(axis=1).iloc[:, 0]
df['VEHICLE_DAMAGE'] = df.filter(regex='VEHICLE_DAMAGE').bfill(axis=1).iloc[:, 0]

to_drop = [
    'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2',
    'CONTRIBUTING_FACTOR_VEHICLE_1', 'CONTRIBUTING_FACTOR_VEHICLE_2', 'CONTRIBUTING_FACTOR_VEHICLE_3',
    'CONTRIBUTING_FACTOR_VEHICLE_4', 'CONTRIBUTING_FACTOR_VEHICLE_5', 
    'VEHICLE_TYPE_CODE_1', 'VEHICLE_TYPE_CODE_2', 'VEHICLE_TYPE_CODE_3', 'VEHICLE_TYPE_CODE_4', 'VEHICLE_TYPE_CODE_5',
    'VEHICLE_DAMAGE_1', 'VEHICLE_DAMAGE_2', 'VEHICLE_DAMAGE_3'
]

df.drop(columns=to_drop, inplace=True)

In [24]:
df.shape

(558724, 29)

## 4. Handle columns with too many missing values

In [25]:
df.isna().mean().pipe(lambda x: x[x > 0.95])

PUBLIC_PROPERTY_DAMAGE_TYPE    0.997455
dtype: float64

In [26]:
to_drop = [
    'PUBLIC_PROPERTY_DAMAGE_TYPE'
]

df.drop(columns=to_drop, inplace=True)

## 5. Create the target variable

- The total number of people injured or killed in an accident will be summed up.
- Location data will be important, so any collision without any location data will be dropped.

In [27]:
df[['LATITUDE', 'LONGITUDE']].mask(lambda x: x == 0).isna().mean()

LATITUDE     0.032198
LONGITUDE    0.032198
dtype: float64

In [28]:
to_drop = [
    'NUMBER_OF_PERSONS_INJURED', 'NUMBER_OF_PERSONS_KILLED',
    'NUMBER_OF_PEDESTRIANS_INJURED', 'NUMBER_OF_PEDESTRIANS_KILLED',
    'NUMBER_OF_CYCLIST_INJURED', 'NUMBER_OF_CYCLIST_KILLED',
    'NUMBER_OF_MOTORIST_INJURED', 'NUMBER_OF_MOTORIST_KILLED'
]

df[['LATITUDE', 'LONGITUDE']] = df[['LATITUDE', 'LONGITUDE']].mask(lambda x: x == 0)
df.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df['CASUALTY'] = df[to_drop].sum(1).gt(0).astype(int)

df.drop(columns=to_drop, inplace=True)

### 5.1. See other columns

In [29]:
df.columns.difference(crashes.columns)

Index(['CASUALTY', 'CONTRIBUTING_FACTOR', 'DRIVER_LICENSE_JURISDICTION',
       'DRIVER_LICENSE_STATUS', 'DRIVER_SEX', 'POINT_OF_IMPACT', 'PRE_CRASH',
       'PUBLIC_PROPERTY_DAMAGE', 'STATE_REGISTRATION', 'TRAVEL_DIRECTION',
       'VEHICLE_DAMAGE', 'VEHICLE_MAKE', 'VEHICLE_OCCUPANTS', 'VEHICLE_TYPE',
       'VEHICLE_TYPE_CODE', 'VEHICLE_YEAR'],
      dtype='object')

In [30]:
df.columns.difference(vehicles.columns)

Index(['CASUALTY', 'CONTRIBUTING_FACTOR', 'LATITUDE', 'LONGITUDE',
       'VEHICLE_TYPE_CODE'],
      dtype='object')

In [31]:
df.isna().mean()

CRASH_DATE                     0.000000
CRASH_TIME                     0.000000
LATITUDE                       0.000000
LONGITUDE                      0.000000
COLLISION_ID                   0.000000
STATE_REGISTRATION             0.040081
VEHICLE_TYPE                   0.027875
VEHICLE_MAKE                   0.604504
VEHICLE_YEAR                   0.606518
TRAVEL_DIRECTION               0.569075
VEHICLE_OCCUPANTS              0.583943
DRIVER_SEX                     0.638042
DRIVER_LICENSE_STATUS          0.654958
DRIVER_LICENSE_JURISDICTION    0.651934
PRE_CRASH                      0.296650
POINT_OF_IMPACT                0.572938
VEHICLE_DAMAGE                 0.575901
PUBLIC_PROPERTY_DAMAGE         0.553555
VEHICLE_TYPE_CODE              0.003270
CONTRIBUTING_FACTOR            0.002809
CASUALTY                       0.000000
dtype: float64

## 6. Dummify License

In [32]:
df.groupby(df['STATE_REGISTRATION'].isin(['NY']))['CASUALTY'].agg(['size', 'mean'])

,size,mean
STATE_REGISTRATION,,
False,128665,0.144507
True,412069,0.133174


In [33]:
df.groupby(df['DRIVER_LICENSE_JURISDICTION'].eq('NY'))['CASUALTY'].agg(['size', 'mean'])

,size,mean
DRIVER_LICENSE_JURISDICTION,,
False,392089,0.126913
True,148645,0.159501


In [34]:
df.groupby(df['DRIVER_LICENSE_JURISDICTION'].isin(['NY', 'NJ', 'CT']))['CASUALTY'].agg(['size', 'mean'])

,size,mean
DRIVER_LICENSE_JURISDICTION,,
False,365982,0.128042
True,174752,0.152267


## 7. Drop unnecessary columns

There are a lot of object dtype columns that could be dummified or just dropped. Let's see if any of them are worth keeping.

In [35]:
df.groupby('DRIVER_LICENSE_STATUS')['CASUALTY'].agg(['size', 'mean'])

,size,mean
DRIVER_LICENSE_STATUS,,
Licensed,183212,0.148265
Permit,1088,0.305147
Unlicensed,2276,0.464851


In [36]:
df.groupby('PRE_CRASH')['CASUALTY'].agg(['size', 'mean'])

,size,mean
PRE_CRASH,,
Avoiding Object in Roadway,579,0.151986
Backing,13366,0.096962
Changing Lanes,18482,0.061682
Entering Parked Position,6866,0.063647
Going Straight Ahead,174636,0.170761
Making Left Turn,24732,0.270540
Making Left Turn on Red,182,0.269231
Making Right Turn,28276,0.104718
Making Right Turn on Red,150,0.146667


In [37]:
df.groupby('VEHICLE_DAMAGE')['CASUALTY'].agg(['size', 'mean']).sort_index()

,size,mean
VEHICLE_DAMAGE,,
Center Back End,14616,0.218117
Center Front End,21221,0.309505
Demolished,107,0.551402
Left Front Bumper,22897,0.124820
Left Front Quarter Panel,18330,0.091162
Left Rear Bumper,10786,0.083256
Left Rear Quarter Panel,14347,0.058967
Left Side Doors,14058,0.110115
No Damage,33521,0.302706


In [38]:
df.groupby('PUBLIC_PROPERTY_DAMAGE')['CASUALTY'].agg(['size', 'mean'])

,size,mean
PUBLIC_PROPERTY_DAMAGE,,
N,215426,0.156694
Unspecified,25217,0.146964
Y,765,0.245752


In [39]:
df.groupby('POINT_OF_IMPACT')['CASUALTY'].agg(['size', 'mean'])

,size,mean
POINT_OF_IMPACT,,
Center Back End,18763,0.222619
Center Front End,26719,0.355665
Demolished,55,0.309091
Left Front Bumper,27902,0.155043
Left Front Quarter Panel,19606,0.107977
Left Rear Bumper,11829,0.081664
Left Rear Quarter Panel,15297,0.063738
Left Side Doors,14708,0.138020
No Damage,7575,0.259538


In [40]:
df['VEHICLE_TYPE_CODE'] = df['VEHICLE_TYPE_CODE'].str.lower()
df.groupby('VEHICLE_TYPE_CODE')['CASUALTY'].agg(['size', 'mean']).sort_values('mean', ascending=False).query('size > 100')

,size,mean
VEHICLE_TYPE_CODE,,
e-bike,283,0.844523
e-scooter,237,0.797468
bike,4205,0.749584
bicycle,354,0.658192
moped,233,0.605150
scooter,157,0.503185
motorscooter,158,0.481013
motorcycle,3474,0.301670
unknown,5543,0.257442


In [41]:
#df.groupby('CONTRIBUTING_FACTOR')['CASUALTY'].agg(['size', 'mean'])
df['CONTRIBUTING_FACTOR'].value_counts(normalize=True, dropna=False)

Unspecified                         0.326438
Driver Inattention/Distraction      0.203914
Other Vehicular                     0.060984
Failure to Yield Right-of-Way       0.041353
Turning Improperly                  0.038357
                                      ...   
Using On Board Navigation Device    0.000022
Texting                             0.000018
80                                  0.000009
1                                   0.000004
Listening/Using Headphones          0.000004
Name: CONTRIBUTING_FACTOR, Length: 62, dtype: float64

In [42]:
to_drop = [
    'STATE_REGISTRATION', 'DRIVER_LICENSE_JURISDICTION', 'DRIVER_LICENSE_STATUS', 
    'VEHICLE_DAMAGE', 'PUBLIC_PROPERTY_DAMAGE', 
    'VEHICLE_TYPE', 'VEHICLE_MAKE', 'VEHICLE_YEAR',
    'TRAVEL_DIRECTION', 'VEHICLE_OCCUPANTS', 'DRIVER_SEX',
    'CONTRIBUTING_FACTOR', 'POINT_OF_IMPACT'
]

df['FRONT_OR_BACK_ENDED'] = df['POINT_OF_IMPACT'].isin(['Overturned', 'Center Front End', 'Demolished', 'Center Back End']).astype(int)

df.drop(columns=to_drop, inplace=True)

In [43]:
df.columns

Index(['CRASH_DATE', 'CRASH_TIME', 'LATITUDE', 'LONGITUDE', 'COLLISION_ID',
       'PRE_CRASH', 'VEHICLE_TYPE_CODE', 'CASUALTY', 'FRONT_OR_BACK_ENDED'],
      dtype='object')

In [44]:
df.groupby('PRE_CRASH')['CASUALTY'].agg(['size', 'mean']).sort_values('mean')

,size,mean
PRE_CRASH,,
Merging,7984,0.048848
Changing Lanes,18482,0.061682
Entering Parked Position,6866,0.063647
Passing,6199,0.068559
Parked,56212,0.071622
Starting from Parking,8153,0.077517
Backing,13366,0.096962
Making Right Turn,28276,0.104718
Starting in Traffic,1712,0.144276


## 8. Dummify Crash columns

In [45]:
in_intersection = [
    'Making Left Turn', 'Making Left Turn on Red', 'Making U Turn', 'Police Pursuit'
]


df['GOING_STRAIGHT_AHEAD'] = df['PRE_CRASH'].eq('Going Straight Ahead').astype(int)
df['IN_INTERSECTION'] = df['PRE_CRASH'].isin(in_intersection).astype(int)
df.drop(columns='PRE_CRASH', inplace=True)

Given the previous `groupby` exercise using `'VEHICLE_TYPE_CODE'`, it's clear that riding a bike or motorcycle, truck or taxi are especially hazardous. So all values containing `'bik'`, `'cyc'`, `'scoo'`, `'moto'`, `'mop'`, `'taxi'` or `'tru'` into a are converted to 1 and all others 0.

In [46]:
def mapping(v):
    if isinstance(v, str):
        if 'bik' in v or 'cyc' in v or 'scoo' in v or 'moto' in v or 'mop' in v or 'taxi' in v or 'tru' in v:
            return 1
        else:
            return 0
    else:
        return 0

In [47]:
to_drop = [
    'VEHICLE_TYPE_CODE', 'CRASH_DATE', 'CRASH_TIME'
]

df['CRASH_DATETIME'] = df['CRASH_DATE'] + ' ' + df['CRASH_TIME']
df['TRUCK_TAXI_BIKE'] = df['VEHICLE_TYPE_CODE'].map(mapping)

df.drop(columns=to_drop, inplace=True)

## 9. Save processed dataframe

In [48]:
def aggfunc(col):
    if col in {'LATITUDE', 'LONGITUDE', 'CRASH_DATETIME'}:
        return 'first'
    else:
        return 'max'
    
aggfuncs = {k: aggfunc(k) for k in df.columns.drop('COLLISION_ID')}
df = df.groupby('COLLISION_ID').agg(aggfuncs).set_index('CRASH_DATETIME')

In [49]:
df.shape

(279345, 7)

In [50]:
df.isna().sum()

LATITUDE                0
LONGITUDE               0
CASUALTY                0
FRONT_OR_BACK_ENDED     0
GOING_STRAIGHT_AHEAD    0
IN_INTERSECTION         0
TRUCK_TAXI_BIKE         0
dtype: int64

In [51]:
df.dtypes

LATITUDE                float64
LONGITUDE               float64
CASUALTY                  int32
FRONT_OR_BACK_ENDED       int32
GOING_STRAIGHT_AHEAD      int32
IN_INTERSECTION           int32
TRUCK_TAXI_BIKE           int64
dtype: object

In [52]:
df.to_csv('../data/processed_data/data_0.csv')